# Environment Setup

In [ ]:
!git clone https://github.com/supplient/bachelor_design.git
import os
os.chdir("bachelor_design")
!pwd

In [ ]:
!git checkout bert_replace
!git pull

In [ ]:
locale = False
try:
    import google.colab
    google.colab.drive.mount('/gdrive')
except ModuleNotFoundError:
    locale = True

In [ ]:
addh = "/gdrive/My Drive"
if locale:
    addh = "/mnt/d/My Drive"
    print("[Locale] Using address head: " + addh)
else:
    print("[Colab] Using address head: " + addh)

# Prepare Input

In [ ]:
import preprocess

In [ ]:
import pickle
tag_vocab = None
with open(addh + "/Graduation/Model/tag_vocab.pkl", "rb") as fd:
    tag_vocab = pickle.load(fd)

In [ ]:
SEQ_LEN = 512

test_char_seqs, test_tag_seqs = preprocess.load_file(
    addh + "/Graduation/Data/test_dict.txt")
test_token_id_seqs, test_segment_seqs, test_tag_id_seqs, tag_vocab = preprocess.preprocess(
    test_char_seqs, 
    test_tag_seqs,
    addh + "/Graduation/BERT/multi_cased_L-12_H-768_A-12/vocab.txt",
    SEQ_LEN=512,
    tag_vocab=tag_vocab
    )

# Build Model

In [ ]:
import keras

In [ ]:
input_token = keras.layers.Input(shape=(SEQ_LEN,))
input_segment = keras.layers.Input(shape=(SEQ_LEN,))

bert_output = bert_model([input_token, input_segment])

output = keras.layers.Dense(len(tag_vocab), activation=keras.activations.softmax)(bert_output)

model = keras.models.Model([input_token, input_segment], output)

# Predict

In [ ]:
predict_tag_id_seqs = model.predict(
    [test_token_id_seqs, test_segment_seqs],
    batch_size=8,
    verbose=1
)

In [ ]:
import analyze
analyze(
    test_char_seqs,
    test_tag_seqs,
    predict_tag_id_seqs,
    tag_vocab
)